In [1]:
import PyPDF2 
import textract
import re
from nltk import sent_tokenize
import pandas as pd
from urllib.request import urlopen
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from PIL import Image
import pandas as pd 


# Download ntlk
# nltk.download()

def get_content(website, dest_file=""):
    """
    README: This function is used to copy text (html type <p>) into an output file. 
        If failed, no text will be copied.

    website: the website to grab information.
    dest_file: output file name (and directory). *.txt recommended. Note that new text 
        will be appended behind the old text. If not specified, the text will be printed
        (not recommend).

    In order to split the sentences, package nltk is used herein.
        nltk needs to be downloaded first by using "nltk.download()".
    """

    driver = webdriver.Chrome("/Users/vedantmathur/Downloads/chromedriver")
    try:
        driver.get(website)
        content_element = driver.find_element_by_tag_name("div")
        content_html = content_element.get_attribute("innerHTML")
        soup = BeautifulSoup(content_html, "html.parser")
        p_tags = soup.find_all("p")
        #print(content_html) 
        #print(dest_file) 
        if (dest_file != ""):
          #  print("what") 
          #  print(p_tags) 
            text = open(dest_file, "w")
            totalSentences = [] 
            for p in p_tags:
                #text.write(p.getText())
                sentences = sent_tokenize(p.getText())
                print(sentences)
                for i in range(len(sentences)):
                    text.write(sentences[i])
                    text.write("\n")
                    
                    totalSentences.append(sentences[i]) 
            text.close()
            
            dataFrame = pd.DataFrame(data = {"sentence": totalSentences}) 
            dataFrame.to_csv("data.csv") 
        else:
            for p in p_tags:
                sentences = sent_tokenize(p.getText())
                for i in range(len(sentences)):
                    print(sentences[i])
        driver.close()
    except:
        driver.close()

def read_pdf(filename):
    """
    README: This function is used to read texts from a PDF file into a list of strings.
        Each string in the list is one row of text.
        This is a helper function.

    filename: the file name of *.pdf file (and directory).
    RETURN: a list of strings
    """

    """
    pdfFileObj = open(filename,'rb')  
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    pdfReader.numPages
    pageObj = pdfReader.getPage(3)
    text = pageObj.extractText()
    if (text == ""):
    """
    text = textract.process(filename, method='tesseract', language='eng')
    return str(text).split('\\n')

def steer(source_file, dest_file):
    """
    README: This function is used to read texts from websites referenced in STEER earthquake briefings.
        This function calls read_pdf to locate and extract the websites inside the PDF, 
        then it iterates over websites by calling get_content to obtain the texts.

    source_file: PDF file name (and directory).
    dest_file: output file name (and directory).
    """
    lines = read_pdf(source_file)
    i = 0
    while (i < len(lines)):
        index = lines[i].find("eference") #reference
        if (index != -1):
            break
        i += 1
    while (i < len(lines)):
        #print(i)
        index = lines[i].find("http")
        if (index == -1):
            i += 1
            continue
        url = lines[i][index:]
        url_found = False
        try:
            req = requests.get(url)
        except:
            i += 1
            continue
        if (str(req) == "<Response [200]>"):
            url_found = True
        else:
            url += lines[i+1]
            try:
                req = requests.get(url)
            except:
                i += 1
                continue
            if (str(req) == "<Response [200]>"):
                url_found = True
                
        if (not url_found):
            url = lines[i][index:]
            url.replace(" ", "")
            try:
                req = requests.get(url)
            except:
                i += 1
                continue
            if (str(req) == "<Response [200]>"):
                url_found = True
            else:
                url += lines[i+1]
                url.replace(" ", "")
                try:
                    req = requests.get(url)
                except:
                    i += 1
                    continue
                if (str(req) == "<Response [200]>"):
                    url_found = True
        if (not url_found):
            url = lines[i][index:]
            url.replace(" ", ".")
            try:
                req = requests.get(url)
            except:
                i += 1
                continue
            if (str(req) == "<Response [200]>"):
                url_found = True
            else:
                url += lines[i+1]
                url.replace(" ", ".")
                try:
                    req = requests.get(url)
                except:
                    i += 1
                    continue
                if (str(req) == "<Response [200]>"):
                    url_found = True
                    
        if (not url_found):
            i += 1
            continue
        #print(url)
        get_content(url, dest_file)
        i += 1
        
get_content("https://www.reuters.com/article/us-argentina-quake/magnitude-68-quake-strikes-san-juan-province-argentina-gfz-idUSKBN29O077", "article.txt") 

['Discover Thomson Reuters']
['More']
['By Reuters Staff']
['2 Min Read']
['(Reuters) - An earthquake of magnitude 6.8 struck west-central Argentina on Monday, followed by at least five aftershocks that shook buildings and sent products tumbling off supermarket shelves, but there were no immediate reports of injuries.']
['The quake hit at a depth of 10 km (6 miles), the German Research Center for Geosciences (GFZ) said, with its epicentre located 40 km (25 miles) south of the province of San Juan.']
['The aftershocks ranged in magnitude from 3.5 to 5, the European Mediterranean Seismological Centre (EMSC) said.']
['Posts on social media showed many homes and buildings shaking, with the seismic activity leaving cracks in roads, while glass bottles fell off the shelves in shops.']
['San Juan Governor Sergio Uñac urged people to remain calm following the earthquake and called on them to reach out in case they need any assistance.']
['"Let us put into practice all the measures we have lear

In [8]:
from datetime import datetime
from collections import defaultdict 

from matplotlib import pyplot as plt
import math

data = pd.read_csv("dataalbania.csv") 
chosen = ((data["date"][0]))

def convertToTime(inputString): 
    year = int(inputString[0:4])
    month = int(inputString[5:7])
    day = int(inputString[8:10])
    hour = int(inputString[11:12 + 1])
    minute = int(inputString[14:16])
    seconds = int(inputString[17:19])

    dateObject = datetime(year, month, day, hour, minute, seconds) 
    
    return dateObject

def dateDifference(date1, date2): 
    return divmod((convertToTime(date1) - convertToTime(date2)).total_seconds(), 3600 * 24)

earthquakeLog = pd.read_csv("data/earthquakes_log.csv") 
twitterCSV = pd.read_csv("data/twitter/tweets.csv") 

def processCSV(logFile, twitterFile, country, keywords): 
    frequencies = defaultdict(int) 
    log = pd.read_csv(logFile) 
    tweets = pd.read_csv(twitterFile)
    print()
    tweets = tweets.to_numpy() 
    #print(log)
    logTime = log[log["country"] == country]["rupture_time"][0]
    #print(logTime)
    
#print(twitterCSV.to_numpy())twitterCSV = twitterCSV.to_numpy()
   # print(tweets)
    for x in range(0, len(tweets)): 
       # print(twitterCSV[x][4])
      
       # if(any([keywords[j] in tweets[x][4] for j in range(0, len(keywords))])):
        
       # print("what\n\n\n\n\n")
        differenceTweet = (dateDifference(tweets[x][0], logTime)[0]) 
        frequencies[differenceTweet] += 1
            
    print(frequencies)
    x = frequencies.keys() 
    y = frequencies.values()
    
    plt.plot(x, y)
    plt.show()
   
def processTexas(logFile, twitterFile, keywordCategory, keywords): 
    frequencies = defaultdict(int) 
    log = pd.read_csv(logFile) 
    tweets = pd.read_csv(twitterFile)
    print()
    tweets = tweets.to_numpy() 
    #print(log)
    logTime = "2021-02-10 00:00:00"
    #print(logTime)
    
#print(twitterCSV.to_numpy())twitterCSV = twitterCSV.to_numpy()
   # print(tweets)
    for x in range(0, len(tweets)): 
      #  print(tweets[x][0])
      #  print((tweets[x][3]))
    
        if(any([keywords[j] in tweets[x][2] for j in range(0, len(keywords))])):
       # print("what\n\n\n\n\n")
            differenceTweet = (dateDifference(tweets[x][0], logTime)[0]) 
            frequencies[differenceTweet] += 1
            
    print(frequencies)
    x = frequencies.keys() 
    y = frequencies.values()
    
    plt.plot(x, y)
  #  plt.show()
    plt.xlabel("days after 2/10") 
    plt.ylabel("number of tweet") 
    plt.title(keywordCategory)
    
    plt.savefig(keywordCategory + ".png")
    
    plt.close()
   
texasKeywords = pd.read_csv("data/Texas-Keywords.csv")
texasColumns = (texasKeywords.columns)
texasColumns = texasColumns.to_numpy()
print(texasColumns)
texasKeywords = texasKeywords.to_numpy() 


#print(texasKeywords.transpose())

for x in range(0, len(texasKeywords.transpose())): 
    a = texasKeywords.transpose()[x]
    a = list(a) 
   # print(a) 
    a = filter(lambda x: type(x) == str, a)
    a = list(a)
    
    print(a)
    processTexas("data/earthquakes_log.csv", "data/twitter/tweetdata.csv", texasColumns[x], a) 
    

['Power' 'Water supply' 'Gas' 'Transportation' 'Retail services'
 'Internet provider services' 'Telecommunication' 'School' 'Hospital'
 'Office']
['power plant', 'wind turbine', 'solar panel', 'electricity', 'power outage']

defaultdict(<class 'int'>, {26.0: 1, 24.0: 2, 23.0: 2, 22.0: 2, 20.0: 2, 17.0: 1, 14.0: 2, 13.0: 2, 11.0: 3, 10.0: 10, 9.0: 16, 8.0: 45, 7.0: 43, 6.0: 47, 5.0: 22, 4.0: 1})
['water', 'dam', 'faucet']

defaultdict(<class 'int'>, {28.0: 1, 27.0: 2, 26.0: 2, 25.0: 3, 24.0: 1, 22.0: 1, 20.0: 1, 18.0: 3, 17.0: 4, 16.0: 8, 15.0: 3, 14.0: 7, 13.0: 13, 12.0: 10, 11.0: 19, 10.0: 26, 9.0: 56, 8.0: 114, 7.0: 76, 6.0: 36, 5.0: 12, 4.0: 3})
['pipeline', 'gas', 'heat', 'refinery', 'cold']

defaultdict(<class 'int'>, {30.0: 1, 29.0: 1, 28.0: 2, 27.0: 1, 26.0: 1, 24.0: 1, 23.0: 4, 22.0: 5, 20.0: 3, 19.0: 1, 16.0: 4, 15.0: 2, 14.0: 7, 13.0: 3, 12.0: 3, 11.0: 7, 10.0: 11, 9.0: 25, 8.0: 71, 7.0: 81, 6.0: 100, 5.0: 55, 4.0: 16, 3.0: 5})
['bridge', 'highway', 'road ', 'airport', 'tunne